In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

from toolbox_continu_inzicht import Config, DataAdapter
from toolbox_continu_inzicht.fragility_curves import (
    ChangeCrestHeightFragilityCurveOvertoppingBedlevelFetch,
    FragilityCurveOvertoppingBedlevelFetchMultiple,
    ShiftFragilityCurveOvertoppingBedlevelFetch,
)

In [ ]:
path = Path.cwd() / "data_sets"
config = Config(config_path=path / "demo_fragility_multiple_curves_overtopping_db.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

```yaml
GlobalVariables:
    rootdir: "data_sets"
    moments: [-24,0,24,48]

    FragilityCurveOvertoppingBedlevelFetchMultiple:
        gh_onz_mu: 0.96 # default waarde

DataAdapter:
    slopes:
        type: ci_postgresql_slopes
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    profiles:
        type: ci_postgresql_profiles
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
    
    bedlevel:
        type: ci_postgresql_bedlevelfetch
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"
    
    fragility_curves:
        type: ci_postgresql_fragilitycurves_table
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

    read_fc:
        type: ci_postgresql_fragilitycurves_table
        database: "geoserver"
        schema: "continuinzicht_demo_realtime"

```

In [ ]:
fragility_curves_overtopping = FragilityCurveOvertoppingBedlevelFetchMultiple(
    data_adapter=data_adapter
)
fragility_curves_overtopping.run(
    input=["slopes", "profiles", "bedlevel"],
    output="fragility_curves",
)

In [ ]:
df = fragility_curves_overtopping.df_out
df = df[df["section_id"] == 27]
df

In [ ]:
ids = fragility_curves_overtopping.df_out.section_id.unique()
df = fragility_curves_overtopping.df_out.set_index("hydraulicload")
df = df[df["section_id"] == 27]

In [ ]:
df

In [ ]:
shift_fragility_curves_overtopping = FragilityCurveOvertoppingBedlevelFetchMultiple(
    data_adapter=data_adapter
)
shift_fragility_curves_overtopping.fragility_curve_function = (
    ShiftFragilityCurveOvertoppingBedlevelFetch
)
shift_fragility_curves_overtopping.measure_id = 1
shift_fragility_curves_overtopping.effect = 0.5
shift_fragility_curves_overtopping.run(
    input=["slopes", "profiles", "bedlevel"],
    output="fragility_curves",
)

In [ ]:
df_shift = shift_fragility_curves_overtopping.df_out.set_index("hydraulicload")
df_shift = df_shift[df_shift["section_id"] == 27]

In [ ]:
fig, ax = plt.subplots(1, 1)
id = 27
df[df["section_id"] == id]["failure_probability"].plot(
    color="C0", ls="--", label=f"section {id}", ax=ax
)
df_shift[df_shift["section_id"] == id]["failure_probability"].plot(color="C0", ax=ax)
ax.legend()
ax.set_ylabel("failure probability")

In [ ]:
change_crest_fragility_curves_overtopping = (
    FragilityCurveOvertoppingBedlevelFetchMultiple(data_adapter=data_adapter)
)
change_crest_fragility_curves_overtopping.fragility_curve_function = (
    ChangeCrestHeightFragilityCurveOvertoppingBedlevelFetch
)
shift_fragility_curves_overtopping.measure_id = 2
change_crest_fragility_curves_overtopping.effect = 0.5
change_crest_fragility_curves_overtopping.run(
    input=["slopes", "profiles", "bedlevel"],
    output="fragility_curves",
)

In [ ]:
df_change_crest = change_crest_fragility_curves_overtopping.df_out.set_index(
    "hydraulicload"
)
df_change_crest = df_change_crest[df_change_crest["section_id"] == 27]

In [ ]:
fig, ax = plt.subplots(1, 1)
id = 27
df[df["section_id"] == id]["failure_probability"].plot(
    ls="--", label=f"section {id}", ax=ax, alpha=0.25, color="C0"
)
df_shift[df_shift["section_id"] == id]["failure_probability"].plot(
    ax=ax, ls="-.", color="C0"
)
df_change_crest[df_change_crest["section_id"] == id]["failure_probability"].plot(
    ax=ax, alpha=0.5, color="C0"
)
ax.legend(labels=["original", "shift", "change crest"])
ax.set_ylabel("failure probability")